# Preprocess UCK data for SARS-CoV-2 detection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

## CBC data preprocessing
Read dataframe with CBC results.

In [2]:
features = ['WBC', 'HGB', 'MCV', 'MCHC', 'PLT', '% LYMPH', '% MONO', '% EO', '% BAZO'] # Input features of the severity model

df_morf = pd.read_csv('../datasets/raw/uck-data/tests/Morfologia.csv') # Raw data with ungrouped lab results
df_morf = df_morf[['atrybut_kod', 'pacjent_id', 'badanie_data', 'numer_badania', 'wartosc']] # Columns needed for matching data

df_morf = df_morf.loc[df_morf['atrybut_kod'].isin(features)] # Limit rows to those describing model severity
df_morf['badanie_data'] = pd.to_datetime(df_morf['badanie_data'])
df_morf.reset_index(drop=True, inplace=True)
df_morf

,atrybut_kod,pacjent_id,badanie_data,numer_badania,wartosc
0,HGB,100055,2020-08-07 11:56:00,CN63395434,11.7
1,MCV,100055,2020-08-07 11:56:00,CN63395434,88.7
2,MCHC,100055,2020-08-07 11:56:00,CN63395434,32.5
3,PLT,100055,2020-08-07 11:56:00,CN63395434,176
4,WBC,100055,2020-08-07 11:56:00,CN63395434,6.54
...,...,...,...,...,...
1119823,WBC,99960,2020-11-27 14:49:00,CN66582744,0.80
1119824,% LYMPH,99960,2020-11-27 14:49:00,CN66582744,11.3
1119825,% MONO,99960,2020-11-27 14:49:00,CN66582744,22.5
1119826,% EO,99960,2020-11-27 14:49:00,CN66582744,2.5


Check if all required features were selected.

In [3]:
df_morf['atrybut_kod'].unique()

array(['HGB', 'MCV', 'MCHC', 'PLT', 'WBC', '% LYMPH', '% MONO', '% EO',
       '% BAZO'], dtype=object)

The values of Complete Blood Count features were inserted manually and contain typos. Fix them to be valid numbers. 

In [4]:
import re
for idx, row in df_morf.iterrows():
    col = 'wartosc' # The name of a column with a parameter value
    if str(row[col]):
        num = str(row[col])
        if ',' in num: # Replace commas with dots to match English numeral conventions.
            num = num.replace(',', '.')
        num = re.findall(r"[-+]?\d*\.\d+|\d+", num) # Extract only a number from the cell
        if len(num) > 0: # If the cell contains a number.
            num = num[0]
            df_morf[col].iat[idx] = float(num) # Replace the value in cell with just the number.
        else:
            df_morf[col].iat[idx] = None # If a valid number wasn't found, replace cell contents with None.

Change type of CBC feature value column to numeric. It should run without errors after preprocessing in cell above.

In [5]:
df_morf['wartosc'] = pd.to_numeric(df_morf['wartosc'], downcast="float")

Create a table aggregating records of all features tested in a single CBC test to a single row in a pivot table.

In [6]:
df_cbc = pd.pivot_table(df_morf, index = ['numer_badania', 'pacjent_id', 'badanie_data'], values=['wartosc'], columns=['atrybut_kod'])
df_cbc

wartosc                         \
atrybut_kod                                   % BAZO % EO    % LYMPH % MONO   
numer_badania pacjent_id badanie_data                                         
CN37415815    1640622    2020-02-12 09:28:00     0.8  2.0  22.100000    9.1   
CN39640263    1649146    2019-11-12 08:44:00     0.4  5.3  22.400000    9.9   
CN41703049    189651     2019-10-31 13:12:00     0.2  0.0  25.000000    8.1   
CN42580687    89507      2019-05-28 11:28:00     0.7  1.0  21.200001    6.2   
CN42711457    254279     2020-07-21 08:26:00     0.8  2.9  30.299999   10.1   
...                                              ...  ...        ...    ...   
CN66802904    1766501    2020-12-04 18:21:00     0.6  1.7  17.799999   12.0   
CN66804052    1951400    2020-12-04 19:51:00     0.1  0.3  12.700000    8.0   
CN66804216    1098876    2020-12-04 19:35:00     1.2  0.7  20.799999   10.5   
CN66804331    323440     2020-12-04 19:34:00     0.2  0.0   8.700000    2.3   
CN66804507    2110197    2020-12-04 20:19:00     0.2  1.3  35.000000    5.5   

                                                                           \
atrybut_kod                                    HGB       MCHC         MCV   
numer_badania pacjent_id badanie_data                                       
CN37415815    1640622    2020-02-12 09:28:00  14.6  33.599998   89.099998   
CN39640263    1649146    2019-11-12 08:44:00  16.0  32.900002   93.099998   
CN41703049    189651     2019-10-31 13:12:00  12.8  33.200001   92.300003   
CN42580687    89507      2019-05-28 11:28:00  12.2  35.500000   78.000000   
CN42711457    254279     2020-07-21 08:26:00  12.8  34.299999   91.199997   
...                                            ...        ...         ...   
CN66802904    1766501    2020-12-04 18:21:00  13.0  33.099998   93.099998   
CN66804052    1951400    2020-12-04 19:51:00  10.9  34.500000   85.199997   
CN66804216    1098876    2020-12-04 19:35:00  13.5  34.700001  100.500000   
CN66804331    323440     2020-12-04 19:34:00  12.8  35.299999   85.000000   
CN66804507    2110197    2020-12-04 20:19:00  16.6  35.900002   81.500000   

                                                            
atrybut_kod                                     PLT    WBC  
numer_badania pacjent_id badanie_data                       
CN37415815    1640622    2020-02-12 09:28:00  324.0  12.11  
CN39640263    1649146    2019-11-12 08:44:00  262.0   6.80  
CN41703049    189651     2019-10-31 13:12:00  163.0   5.31  
CN42580687    89507      2019-05-28 11:28:00  308.0   8.39  
CN42711457    254279     2020-07-21 08:26:00  222.0   6.63  
...                                             ...    ...  
CN66802904    1766501    2020-12-04 18:21:00  262.0   8.66  
CN66804052    1951400    2020-12-04 19:51:00  365.0  10.10  
CN66804216    1098876    2020-12-04 19:35:00  230.0   7.21  
CN66804331    323440     2020-12-04 19:34:00  244.0  11.25  
CN66804507    2110197    2020-12-04 20:19:00  274.0   8.54  

[123829 rows x 9 columns]

Change the pivot table to a regular dataframe. Change column names to their English counterparts.

In [7]:
df_cbc = pd.DataFrame(df_cbc.to_records())
df_cbc = df_cbc.rename(columns = {
    "('wartosc', 'WBC')": 'WBC',
    "('wartosc', 'HGB')": 'HGB',
    "('wartosc', 'MCV')": 'MCV',
    "('wartosc', 'MCHC')": 'MCHC',
    "('wartosc', 'PLT')": 'PLT',
    "('wartosc', '% LYMPH')": 'LYT',
    "('wartosc', '% MONO')": 'MOT',
    "('wartosc', '% EO')": 'EOT',
    "('wartosc', '% BAZO')": 'BAT'
})
df_cbc

,numer_badania,pacjent_id,badanie_data,BAT,EOT,LYT,MOT,HGB,MCHC,MCV,PLT,WBC
0,CN37415815,1640622,2020-02-12 09:28:00,0.8,2.0,22.100000,9.1,14.6,33.599998,89.099998,324.0,12.11
1,CN39640263,1649146,2019-11-12 08:44:00,0.4,5.3,22.400000,9.9,16.0,32.900002,93.099998,262.0,6.80
2,CN41703049,189651,2019-10-31 13:12:00,0.2,0.0,25.000000,8.1,12.8,33.200001,92.300003,163.0,5.31
3,CN42580687,89507,2019-05-28 11:28:00,0.7,1.0,21.200001,6.2,12.2,35.500000,78.000000,308.0,8.39
4,CN42711457,254279,2020-07-21 08:26:00,0.8,2.9,30.299999,10.1,12.8,34.299999,91.199997,222.0,6.63
...,...,...,...,...,...,...,...,...,...,...,...,...
123824,CN66802904,1766501,2020-12-04 18:21:00,0.6,1.7,17.799999,12.0,13.0,33.099998,93.099998,262.0,8.66
123825,CN66804052,1951400,2020-12-04 19:51:00,0.1,0.3,12.700000,8.0,10.9,34.500000,85.199997,365.0,10.10
123826,CN66804216,1098876,2020-12-04 19:35:00,1.2,0.7,20.799999,10.5,13.5,34.700001,100.500000,230.0,7.21
123827,CN66804331,323440,2020-12-04 19:34:00,0.2,0.0,8.700000,2.3,12.8,35.299999,85.000000,244.0,11.25


## RT-PCR data preprocessing
Read the data from RT-PCR tests.

In [8]:
df_pcr = pd.read_csv('../datasets/raw/uck-data/TestResult.csv')
df_pcr

,pacjent_id,data_testu,wynik
0,2089489,2020-10-08 23:26:00,nie wykryto
1,1474205,2020-10-16 10:11:00,nie wykryto
2,1942712,2020-06-28 12:40:00,nie wykryto
3,630608,2020-11-10 07:38:00,nie wykryto
4,185736,2020-07-19 13:44:00,nie wykryto
...,...,...,...
34809,2076073,2020-10-30 10:52:00,nie wykryto
34810,2076073,2020-05-19 22:08:00,nie wykryto
34811,2076073,2020-10-19 12:36:00,nie wykryto
34812,2141180,2020-10-01 00:11:00,nie wykryto


Select only patients with definite positive and negative results.

In [9]:
negative = dict.fromkeys(
    ['nie wykryto', 'nie wyrkyto', 'nie  wykryto', 'nie wykryo', 'nire wykryto', 'nie wykrytonie wykryto',
'niwe wykryto', 'nie wykyto', 'nier wykryto', 'nnie wykryto', 'NIE WYKRYTO', 'nei wykryto'], 0)
positive = dict.fromkeys(
    ['DODATNI', 'testowy dodatni', 'WYKRYTO'], 1)


df_pcr['wynik'] = df_pcr['wynik'].replace(negative)
df_pcr['wynik'] = df_pcr['wynik'].replace(positive)
df_pcr = df_pcr[df_pcr['wynik'].isin([0,1])]
df_pcr

,pacjent_id,data_testu,wynik
0,2089489,2020-10-08 23:26:00,0
1,1474205,2020-10-16 10:11:00,0
2,1942712,2020-06-28 12:40:00,0
3,630608,2020-11-10 07:38:00,0
4,185736,2020-07-19 13:44:00,0
...,...,...,...
34809,2076073,2020-10-30 10:52:00,0
34810,2076073,2020-05-19 22:08:00,0
34811,2076073,2020-10-19 12:36:00,0
34812,2141180,2020-10-01 00:11:00,0


Match CBC tests with RT-PCR tests based on patient id.

In [10]:
df_cbc['pacjent_id'] = pd.to_numeric(df_cbc['pacjent_id'],errors='coerce')
df_pcr['pacjent_id'] = pd.to_numeric(df_pcr['pacjent_id'],errors='coerce')

df_cbc = df_cbc[df_cbc['pacjent_id'].notna()]
df_pcr = df_pcr[df_pcr['pacjent_id'].notna()]

df_detection = df_cbc.merge(df_pcr, on='pacjent_id', how='left').drop_duplicates()
df_detection

/tmp/ipykernel_21406/3050920497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pcr['pacjent_id'] = pd.to_numeric(df_pcr['pacjent_id'],errors='coerce')


,numer_badania,pacjent_id,badanie_data,BAT,EOT,LYT,MOT,HGB,MCHC,MCV,PLT,WBC,data_testu,wynik
0,CN37415815,1640622,2020-02-12 09:28:00,0.8,2.0,22.100000,9.1,14.6,33.599998,89.099998,324.0,12.11,2020-10-26 09:30:00,0
1,CN39640263,1649146,2019-11-12 08:44:00,0.4,5.3,22.400000,9.9,16.0,32.900002,93.099998,262.0,6.80,2020-08-31 10:05:00,0
2,CN41703049,189651,2019-10-31 13:12:00,0.2,0.0,25.000000,8.1,12.8,33.200001,92.300003,163.0,5.31,2020-07-21 11:09:00,0
3,CN42580687,89507,2019-05-28 11:28:00,0.7,1.0,21.200001,6.2,12.2,35.500000,78.000000,308.0,8.39,2020-04-19 08:54:00,0
4,CN42711457,254279,2020-07-21 08:26:00,0.8,2.9,30.299999,10.1,12.8,34.299999,91.199997,222.0,6.63,2020-06-17 09:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307848,CN66802904,1766501,2020-12-04 18:21:00,0.6,1.7,17.799999,12.0,13.0,33.099998,93.099998,262.0,8.66,2020-12-04 17:35:00,0
307849,CN66804052,1951400,2020-12-04 19:51:00,0.1,0.3,12.700000,8.0,10.9,34.500000,85.199997,365.0,10.10,2020-12-05 13:52:00,0
307850,CN66804216,1098876,2020-12-04 19:35:00,1.2,0.7,20.799999,10.5,13.5,34.700001,100.500000,230.0,7.21,2020-12-04 20:00:00,0
307851,CN66804331,323440,2020-12-04 19:34:00,0.2,0.0,8.700000,2.3,12.8,35.299999,85.000000,244.0,11.25,2020-12-04 19:31:00,0


Change type of columns with dates to datetime.

In [11]:
df_detection['badanie_data'] = pd.to_datetime(df_detection['badanie_data']).dt.date
df_detection['data_testu'] = pd.to_datetime(df_detection['data_testu']).dt.date
df_detection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307851 entries, 0 to 307852
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   numer_badania  307851 non-null  object 
 1   pacjent_id     307851 non-null  int64  
 2   badanie_data   307851 non-null  object 
 3   BAT            299700 non-null  float32
 4   EOT            299677 non-null  float32
 5   LYT            299555 non-null  float32
 6   MOT            299542 non-null  float32
 7   HGB            307842 non-null  float32
 8   MCHC           307838 non-null  float32
 9   MCV            307838 non-null  float32
 10  PLT            307841 non-null  float32
 11  WBC            307834 non-null  float32
 12  data_testu     306973 non-null  object 
 13  wynik          306973 non-null  object 
dtypes: float32(9), int64(1), object(4)
memory usage: 24.7+ MB


Match the CBC tests with RT-PCR tests for each patient to find a group of patients for whom we can determine if they were infected with SARS-CoV-2 during the CBC test.

In [12]:
time_delta = 1
df_det_1 = df_detection.loc[((df_detection['data_testu'] - df_detection['badanie_data']).dt.days <= time_delta) & ((df_detection['data_testu'] - df_detection['badanie_data']).dt.days >= - time_delta)]
print(f"{time_delta}: {len(df_det_1)}")
print(df_det_1['wynik'].value_counts())

time_delta = 3
df_det_3 = df_detection.loc[((df_detection['data_testu'] - df_detection['badanie_data']).dt.days <= time_delta) & ((df_detection['data_testu'] - df_detection['badanie_data']).dt.days >= - time_delta)]
print(f"{time_delta}: {len(df_det_3)}")
print(df_det_3['wynik'].value_counts())

1: 30358
0    29660
1      698
Name: wynik, dtype: int64
3: 46699
0    45614
1     1085
Name: wynik, dtype: int64


# Patient information preprocessing
Read dataframe with sex and age.

In [13]:
df_patient = pd.read_csv('../datasets/raw/uck-data/Pacjent.csv')
df_patient

,pacjent_id,data_urodzenia,plec
0,1986978,1952-11-13 00:00:00,M
1,2125885,1976-06-06 00:00:00,K
2,2137871,1986-03-23 00:00:00,K
3,2139429,1952-02-02 00:00:00,K
4,1898419,2017-08-02 00:00:00,M
...,...,...,...
25436,1718106,1965-01-12 00:00:00,M
25437,540816,2004-08-04 00:00:00,M
25438,1272628,1964-04-19 00:00:00,M
25439,2123302,2020-02-08 00:00:00,M


Convert the column with date to datetime and column with sex to boolean values.

In [14]:
df_patient['data_urodzenia'] = pd.to_datetime(df_patient['data_urodzenia']).dt.date
df_patient['plec'] = df_patient['plec'].replace({'K': 0, 'M': 1})

Merge with severity dataframe on patient id.

In [15]:
df_full = df_det_1.merge(df_patient, on='pacjent_id', how='left').drop_duplicates()
df_full

,numer_badania,pacjent_id,badanie_data,BAT,EOT,LYT,MOT,HGB,MCHC,MCV,PLT,WBC,data_testu,wynik,data_urodzenia,plec
0,CN45508940,586684,2020-06-12,1.0,1.4,23.600000,7.5,12.4,34.599998,89.900002,333.0,5.89,2020-06-13,0,1967-11-07,0
1,CN54880603,96819,2020-08-21,0.7,2.0,33.700001,8.0,14.9,32.799999,86.300003,237.0,5.97,2020-08-21,0,1936-05-19,0
2,CN55708093,1793886,2020-07-21,0.8,1.1,32.900002,7.1,12.7,33.900002,82.099998,350.0,8.73,2020-07-21,0,1976-01-30,0
3,CN56182992,1765968,2020-07-28,0.9,1.1,38.099998,9.5,13.7,34.900002,88.900002,280.0,5.27,2020-07-28,0,1979-01-12,0
4,CN56463538,600328,2020-07-06,0.8,1.6,22.200001,7.5,13.4,35.700001,84.699997,263.0,6.30,2020-07-06,0,1946-02-27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30353,CN66802799,862239,2020-12-04,0.6,4.0,23.200001,5.7,13.5,35.000000,94.599998,279.0,12.18,2020-12-04,0,1950-04-21,1
30354,CN66802904,1766501,2020-12-04,0.6,1.7,17.799999,12.0,13.0,33.099998,93.099998,262.0,8.66,2020-12-04,0,1941-12-04,1
30355,CN66804052,1951400,2020-12-04,0.1,0.3,12.700000,8.0,10.9,34.500000,85.199997,365.0,10.10,2020-12-05,0,1931-03-11,1
30356,CN66804216,1098876,2020-12-04,1.2,0.7,20.799999,10.5,13.5,34.700001,100.500000,230.0,7.21,2020-12-04,0,1956-04-21,1


Calculate age.

In [16]:
df_full['Age'] = (df_full['data_testu'] - df_full['data_urodzenia']).dt.days
df_full['Age'] = df_full['Age'].apply(lambda x: int(x)//365)

Discard underage patients.

In [17]:
df_full = df_full[(df_full['Age']>=18) | (df_full['Age']==0)]

Keep only the most recent result for a patient.

In [18]:
df_full = df_full.sort_values('badanie_data').drop_duplicates('pacjent_id',keep='last')

Drop redundant columns. Rename column with test result to target.

In [19]:
df_full = df_full.drop(['numer_badania', 'pacjent_id', 'badanie_data', 'data_testu', 'data_urodzenia'], axis=1)
df_full = df_full.rename(columns = {'wynik': 'target', 'plec': 'Sex'})
df_full.columns

Index(['BAT', 'EOT', 'LYT', 'MOT', 'HGB', 'MCHC', 'MCV', 'PLT', 'WBC',
       'target', 'Sex', 'Age'],
      dtype='object')

In [20]:
pd.pivot_table(df_full, index=['target'], values=['BAT', 'EOT', 'LYT', 'MOT', 'HGB', 'MCHC', 'MCV', 'PLT', 'WBC',
       'Age', 'Sex'], aggfunc='count')

,Age,BAT,EOT,HGB,LYT,MCHC,MCV,MOT,PLT,Sex,WBC
target,,,,,,,,,,,
0,16989,16904,16903,16989,16904,16988,16988,16903,16988,16989,16987
1,463,460,460,463,460,463,463,460,463,463,463


Save processed dataframe.

In [21]:
df_full.to_csv('../datasets/processed/auxiliary/uck_detection1.csv')

## Create balanced dataset (no. of positive cases = no. of negative cases).

In [25]:
import pandas as pd
df = pd.read_csv('../datasets/processed/auxiliary/uck_detection1.csv')
samples_count = len(df.loc[df['target']==1])

df_neg = df.dropna()
df_neg = df_neg.loc[df_neg['target']==0]
df_neg = df_neg.sample(n=samples_count)
df_pos = df.loc[df['target']==1]
df_uck = df_neg.append(df_pos)
df_uck.to_csv('../datasets/processed/auxiliary/balanced_uck_detection1.csv')
df_uck['target'].value_counts()

1    463
0    463
Name: target, dtype: int64